# Data Science for prediction machine:

The data in the database came somewhat scrambled

Problems with the original data:
1. Incomplete (some values were missing in some rows)
2. There are ',' inside some of the string values (this is a comma separated list)
3. The dataset came with both parquet and csv files

The first approach was to use the csv data loaded to python with pandas. The problem was that each year had more than 1.5 to 2.5Gb of data *each* so no laptop with 16Gb of ram could load such a dataset **in memory**. This lead to a clear path, use chunking (with the help of [dask](https://www.dask.org/)). Unfortunately (or fortunately, as well see next) this did not work, the "read" dataframe did not contain all of the information. 

In trying to figure out if dask or the csv was the problem we tried to merge the data from 2 different years and get some data from it with linux tools (it seems that python does not like to parse and write more than 10 million records) such as [awk](https://en.wikipedia.org/wiki/AWK), [uniq](https://en.wikipedia.org/wiki/Uniq), etc... What we found was that some of the string data contained the same character that was used to as the separator in the file (','). 

So, after much work trying to wrangle the data in the csvs to no avail (no parser did the job correctly) we tried to see what these [.parquet](https://parquet.apache.org/) files were. As it seems they are a columnar file (from apache) that does not store all of the data as strings. What this does is reduce the size need to store the data manyfold. To give an example our dataset consisting of two years wasted 3.6Gb of space as csv but has parquet occupied just under 400Mb. Nothing is a silver bullet tho and, as a last gripe by the gods against me, while merging the two parquet files I found that the values of one single column were set as `int64` in one table and `float64` in another... Fortunately this had a decently straightforward solution (casting one of the tables according to the schema of the other).

We can **finally** load the data into memory to see what it is...

# Todo:

1. Understand what columns are import
2. Remove the rest
3. Save the new file and send it to BigQuery

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Lets load the dataset:
flights = pd.read_parquet("../../dataset/flights.parquet")

In [3]:
# And see what columns we are working with:
print(flights.columns.values)

flights.describe(exclude=np.number)

['FlightDate' 'Airline' 'Origin' 'Dest' 'Cancelled' 'Diverted'
 'CRSDepTime' 'DepTime' 'DepDelayMinutes' 'DepDelay' 'ArrTime'
 'ArrDelayMinutes' 'AirTime' 'CRSElapsedTime' 'ActualElapsedTime'
 'Distance' 'Year' 'Quarter' 'Month' 'DayofMonth' 'DayOfWeek'
 'Marketing_Airline_Network' 'Operated_or_Branded_Code_Share_Partners'
 'DOT_ID_Marketing_Airline' 'IATA_Code_Marketing_Airline'
 'Flight_Number_Marketing_Airline' 'Operating_Airline'
 'DOT_ID_Operating_Airline' 'IATA_Code_Operating_Airline' 'Tail_Number'
 'Flight_Number_Operating_Airline' 'OriginAirportID' 'OriginAirportSeqID'
 'OriginCityMarketID' 'OriginCityName' 'OriginState' 'OriginStateFips'
 'OriginStateName' 'OriginWac' 'DestAirportID' 'DestAirportSeqID'
 'DestCityMarketID' 'DestCityName' 'DestState' 'DestStateFips'
 'DestStateName' 'DestWac' 'DepDel15' 'DepartureDelayGroups' 'DepTimeBlk'
 'TaxiOut' 'WheelsOff' 'WheelsOn' 'TaxiIn' 'CRSArrTime' 'ArrDelay'
 'ArrDel15' 'ArrivalDelayGroups' 'ArrTimeBlk' 'DistanceGroup'
 'DivAirportL

/tmp/ipykernel_699/1421152036.py:4: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  flights.describe(exclude=np.number)


,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,IATA_Code_Marketing_Airline,Operating_Airline,IATA_Code_Operating_Airline,Tail_Number,OriginCityName,OriginState,OriginStateName,DestCityName,DestState,DestStateName,DepTimeBlk,ArrTimeBlk
count,10390189,10390189,10390189,10390189,10390189,10390189,10390189,10390189,10390189,10390189,10390189,10340091,10390189,10390189,10390189,10390189,10390189,10390189,10390189,10390189
unique,577,22,380,380,2,2,10,15,10,22,22,6015,374,53,53,374,53,53,19,19
top,2021-11-28 00:00:00,Southwest Airlines Co.,ATL,ATL,False,False,AA,WN,AA,WN,WN,N480HA,"Chicago, IL",TX,Texas,"Chicago, IL",TX,Texas,0800-0859,1600-1659
freq,21331,1796565,496984,496952,10155979,10364997,2717901,1796565,2717901,1796565,1796565,4474,563572,1125459,1125459,563552,1125337,1125337,733885,678128
first,2021-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,2022-07-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Some of these are obviously important while other are the exact opposite:

Very Important:
1. FlightDate
2. Airline
3. Origin
4. Dest
5. Cancelled
6. Diverted
7. CRSDepTime (scheduled dep time)
8. DepTime (actual dep time)
9. DepDelay
10. CRSArrTime (scheduled arrival time)
11. ArrTime (actual arrival time)
12. ArrDelay
13. Flight_Number_Operating_Airline

In [4]:
# We can separate these out:

important_columns = ["FlightDate",
                     "Airline",
                     "Operating_Airline",
                     "Origin",
                     "Dest",
                     "AirTime",
                     "Cancelled",
                     "Diverted",
                     "CRSDepTime",
                     "DepTime",
                     "DepDelay",
                     "CRSArrTime",
                     "ArrTime",
                     "ArrDelay",
                     "OriginAirportID",
                     "DestAirportID",
                     "Tail_Number",
                     "CRSElapsedTime",
                     "Flight_Number_Operating_Airline"]

flight_data = flights[important_columns]

flight_data.columns.values

array(['FlightDate', 'Airline', 'Operating_Airline', 'Origin', 'Dest',
       'AirTime', 'Cancelled', 'Diverted', 'CRSDepTime', 'DepTime',
       'DepDelay', 'CRSArrTime', 'ArrTime', 'ArrDelay', 'OriginAirportID',
       'DestAirportID', 'Tail_Number', 'CRSElapsedTime',
       'Flight_Number_Operating_Airline'], dtype=object)

In [5]:
# And save as new file

flight_data.to_parquet('../../dataset/flight-data.parquet')

# Needed links

1. https://stackoverflow.com/questions/34682828/extracting-specific-selected-columns-to-new-dataframe-as-a-copy
2. https://pandas.pydata.org/pandas-docs/version/1.1/reference/api/pandas.DataFrame.to_parquet.html